## Pull Markit data draft

This file aims to record attempts to pull the markit data from the WRDS platform.

## How to replicate the CDS data?

First I want to plan on how to construct and replicate the data and then start collecting from Markit. 

### Portfolio Construction and Calculations:
We want to sort CDS contracts into **20 portfolios** based on their **credit spreads**.

1. Computing CDS Returns:
   - Use the formula for **CDS excess returns**:
     $$
     \text{CDS Return}_t = \frac{\text{CDS Spread}_{t-1}}{250} + \Delta \text{CDS Spread}_t \times \text{Risky Duration}_{t-1}
     $$
     where:
     - The **first term** represents the carry from selling CDS protection.
     - The **second term** is the capital gain/loss from changes in CDS spreads.

2. Risky Duration Calculation:
   - Compute **risky duration (RD)** for CDS contracts:
     $$
     RD_t = \frac{1}{4} \sum_{j=1}^{4M} e^{-j\lambda/4} e^{-j r_{j/4} / 4}
     $$
     where:
     - $\lambda$ is the hazard rate estimated from CDS spreads.
     - $r_{j/4}$ is the risk-free rate at quarter $j/4$.

## Explore Markit Dataset


In [62]:
import wrds
WRDS_USERNAME="ygxu"

db = wrds.Connection(wrds_username = WRDS_USERNAME)

Loading library list...
Done


In [ ]:
lib_list = db.list_libraries()

In [4]:
if "markit" in lib_list:
    print("Markit is in the list!")
else:
    print("Markit is not in the list.")

Markit is in the list!


In [15]:
lst = db.list_tables(library="markit")

In [16]:
for i in lst:
    print(i)

cds2001
cds2002
cds2003
cds2004
cds2005
cds2006
cds2007
cds2008
cds2009
cds2010
cds2011
cds2012
cds2013
cds2014
cds2015
cds2016
cds2017
cds2018
cds2019
cds2020
cds2021
cds2022
cds2023
cds2024
cds2025
cdslookup
cdxcomps
cdxcomps_chars
cdxconst
cdxconst_chars
chars
isdatt
itraxxascomps
itraxxascomps_chars
itraxxasconst
itraxxasconst_chars
itraxxeucomps
itraxxeucomps_chars
itraxxeuconst
itraxxeuconst_chars
itraxxsocomps
itraxxsocomps_chars
itraxxsoconst
itraxxsoconst_chars
red_chars
redent
redentaucccy
redentcorpact
redentcredauc
redentisda
redentrating
redentsuccession
redindex
redlookup
redobl
redoblindxconst
redobllookup
redoblprospectus
redoblrefentity


**For now, we try 2001 first to see if the data work**

In [6]:
db.describe_table(library="markit",table="cds2001")

Approximately 2078458 rows in markit.cds2001.


,name,nullable,type,comment
0,date,True,DATE,Date
1,batch,True,VARCHAR(3),Batch
2,ticker,True,VARCHAR(25),Ticker
3,shortname,True,VARCHAR(100),Short Name
4,redcode,True,VARCHAR(6),Red Code
...,...,...,...,...
158,irdv01,True,DOUBLE PRECISION,Change in Mark to Market from Basis Point in I...
159,rec01,True,DOUBLE PRECISION,Change in Mark to Market from Basis Point in R...
160,dp,True,DOUBLE PRECISION,Default Probability
161,jtd,True,DOUBLE PRECISION,Jump to Default


In [65]:
query="""
    SELECT ticker, redcode, DATE_TRUNC('month', date) AS month, MIN(date) AS first_trade_date, AVG(parspread) AS midspread
    FROM markit.cds2001
    WHERE tenor = '5Y'
    GROUP BY month, ticker, redcode
    """
#db.raw_sql(query)
cds01_final_df = db.raw_sql(query)

In [54]:
query ="""
    SELECT date, ticker, shortname, redcode, sector, region, country,
    parspread, convspreard, upfront,
    cdsrealrecovery, cdsassumedrecovery,
    curveliquidityscore, compositedepth5y, primarydepthcontributed,
    bidasktype5y
    tenor, irdv01, rec01
    FROM markit.cds2001
    WHERE tenor = '5Y'
    LIMIT 10
"""

db.raw_sql(query)

,date,ticker,shortname,redcode,sector,region,country,parspread,convspreard,upfront,cdsrealrecovery,cdsassumedrecovery,curveliquidityscore,compositedepth5y,primarydepthcontributed,tenor,irdv01,rec01
0,2001-01-09,AA,Alcoa Inc.,014B98,Basic Materials,N.Amer,United States,0.004483,None,None,0.2,None,None,3.0,None,None,None,None
1,2001-01-09,AA,Alcoa Inc.,014B98,Basic Materials,N.Amer,United States,0.004192,None,None,0.2,None,None,3.0,None,None,None,None
2,2001-01-10,AA,Alcoa Inc.,014B98,Basic Materials,N.Amer,United States,0.004150,None,None,0.2,None,None,3.0,None,None,None,None
3,2001-01-10,AA,Alcoa Inc.,014B98,Basic Materials,N.Amer,United States,0.003880,None,None,0.2,None,None,3.0,None,None,None,None
4,2001-01-11,AA,Alcoa Inc.,014B98,Basic Materials,N.Amer,United States,0.004150,None,None,0.2,None,None,3.0,None,None,None,None
5,2001-01-11,AA,Alcoa Inc.,014B98,Basic Materials,N.Amer,United States,0.003880,None,None,0.2,None,None,3.0,None,None,None,None
6,2001-01-12,AA,Alcoa Inc.,014B98,Basic Materials,N.Amer,United States,0.004150,None,None,0.2,None,None,3.0,None,None,None,None
7,2001-01-12,AA,Alcoa Inc.,014B98,Basic Materials,N.Amer,United States,0.003880,None,None,0.2,None,None,3.0,None,None,None,None
8,2001-01-15,AA,Alcoa Inc.,014B98,Basic Materials,N.Amer,United States,0.004150,None,None,0.2,None,None,3.0,None,None,None,None
9,2001-01-15,AA,Alcoa Inc.,014B98,Basic Materials,N.Amer,United States,0.003880,None,None,0.2,None,None,3.0,None,None,None,None


In [66]:
query = """
WITH first_trade_dates AS (
    SELECT 
        ticker, 
        redcode, 
        DATE_TRUNC('month', date) AS month, 
        MIN(date) AS first_trade_date
    FROM markit.cds2001
    GROUP BY month, ticker, redcode
)

SELECT 
    t1.*
FROM markit.cds2001 t1
JOIN first_trade_dates t2
ON t1.ticker = t2.ticker 
AND t1.redcode = t2.redcode 
AND t1.date = t2.first_trade_date
WHERE t1.tenor = '5Y';
"""

cds01_df = db.raw_sql(query)

In [59]:
cds01_final_df

,ticker,redcode,month,first_trade_date,midpspread
0,EMN,29EB75,2001-08-01 00:00:00-04:00,2001-08-10,0.011089
1,CROATI,248DEC,2001-08-01 00:00:00-04:00,2001-08-01,0.016329
2,TIN,8FGHEE,2001-05-01 00:00:00-04:00,2001-05-01,0.011838
3,LNC,5I759F,2001-11-01 00:00:00-05:00,2001-11-01,0.004350
4,CITOH,4ABC88,2001-07-01 00:00:00-04:00,2001-07-02,0.019080
...,...,...,...,...,...
6782,DOL,27BC65,2001-05-01 00:00:00-04:00,2001-05-01,0.022192
6783,PD,7I898B,2001-10-01 00:00:00-04:00,2001-10-01,0.032467
6784,DEXBB-FSAInc,348DFY,2001-04-01 00:00:00-05:00,2001-04-23,0.003267
6785,TGT,8FD73G,2001-07-01 00:00:00-04:00,2001-07-02,0.003064


## Calculate returns

In [69]:
df = cds01_final_df
df

,ticker,redcode,month,first_trade_date,midspread
0,EMN,29EB75,2001-08-01 00:00:00-04:00,2001-08-10,0.011089
1,CROATI,248DEC,2001-08-01 00:00:00-04:00,2001-08-01,0.016329
2,TIN,8FGHEE,2001-05-01 00:00:00-04:00,2001-05-01,0.011838
3,LNC,5I759F,2001-11-01 00:00:00-05:00,2001-11-01,0.004350
4,CITOH,4ABC88,2001-07-01 00:00:00-04:00,2001-07-02,0.019080
...,...,...,...,...,...
6782,DOL,27BC65,2001-05-01 00:00:00-04:00,2001-05-01,0.022192
6783,PD,7I898B,2001-10-01 00:00:00-04:00,2001-10-01,0.032467
6784,DEXBB-FSAInc,348DFY,2001-04-01 00:00:00-05:00,2001-04-23,0.003267
6785,TGT,8FD73G,2001-07-01 00:00:00-04:00,2001-07-02,0.003064


In [71]:
df

,ticker,redcode,month,first_trade_date,midspread
0,EMN,29EB75,2001-08-01 00:00:00-04:00,2001-08-10,0.011089
1,CROATI,248DEC,2001-08-01 00:00:00-04:00,2001-08-01,0.016329
2,TIN,8FGHEE,2001-05-01 00:00:00-04:00,2001-05-01,0.011838
3,LNC,5I759F,2001-11-01 00:00:00-05:00,2001-11-01,0.004350
4,CITOH,4ABC88,2001-07-01 00:00:00-04:00,2001-07-02,0.019080
...,...,...,...,...,...
6782,DOL,27BC65,2001-05-01 00:00:00-04:00,2001-05-01,0.022192
6783,PD,7I898B,2001-10-01 00:00:00-04:00,2001-10-01,0.032467
6784,DEXBB-FSAInc,348DFY,2001-04-01 00:00:00-05:00,2001-04-23,0.003267
6785,TGT,8FD73G,2001-07-01 00:00:00-04:00,2001-07-02,0.003064


In [74]:
df["prev_spread"] = df.groupby("ticker")["midspread"].shift(1)
df_first_term = df.copy()

## Calculate RD


In [ ]:
# Here I have put the data in the data_manual folder
# it can be downloaed automatically in the future.
import pandas as pd 
file_path = "../data_manual/feds200628.csv"
df = pd.read_csv(file_path,skiprows=9)
df["Date"] = pd.to_datetime(df["Date"])

In [82]:
import numpy as np
from scipy.interpolate import interp1d

rf_columns = ["SVENY01", "SVENY02", "SVENY03", "SVENY05", "SVENY07", "SVENY10"]
maturities = [1, 2, 3, 5, 7, 10]

def compute_hazard_rate(cds_spread, lgd=0.60):
    """
    Computes the hazard rate (λ) from the CDS spread.
    """
    cds_spread = cds_spread / 10000  # Convert bps to decimal
    return 4 * np.log(1 + cds_spread / (4 * lgd))

def compute_risky_duration(cds_spread, rf_rates, maturities, lgd=0.60, M=5):
    """
    Computes the Risky Duration (RD) for a CDS contract.

    Parameters:
    - cds_spread: The CDS spread in basis points (bps).
    - rf_rates: A list of interpolated risk-free rates for quarterly maturities.
    - maturities: A list of corresponding maturities.
    - lgd: Loss given default (default is 60%).
    - M: Maturity of the CDS in years (default is 5 years).

    Returns:
    - Risky duration (RD)
    """
    # Compute hazard rate λ
    hazard_rate = compute_hazard_rate(cds_spread, lgd)

    # Interpolate risk-free rates for quarterly maturities
    quarterly_maturities = np.arange(0.25, 4 * M / 4 + 0.25, 0.25)
    rf_curve = interp1d(maturities, rf_rates, kind='cubic', fill_value="extrapolate")
    interpolated_rf_rates = rf_curve(quarterly_maturities)

    # Compute RD sum using hazard rate and risk-free rates
    RD_sum = sum(
        np.exp(-j * hazard_rate / 4) * np.exp(-j * interpolated_rf_rates[j-1] / 4)
        for j in range(1, 4*M + 1)
    )

    # Final RD computation
    return (1/4) * RD_sum

In [83]:
df_first_term.head(5)

,ticker,redcode,month,first_trade_date,midspread,prev_spread
0,EMN,29EB75,2001-08-01 00:00:00-04:00,2001-08-10,0.011089,NaN
1,CROATI,248DEC,2001-08-01 00:00:00-04:00,2001-08-01,0.016329,NaN
2,TIN,8FGHEE,2001-05-01 00:00:00-04:00,2001-05-01,0.011838,NaN
3,LNC,5I759F,2001-11-01 00:00:00-05:00,2001-11-01,0.004350,NaN
4,CITOH,4ABC88,2001-07-01 00:00:00-04:00,2001-07-02,0.019080,NaN


In [84]:
df_yield = df.copy()

In [85]:
df_first_term["first_trade_date"] = pd.to_datetime(df_first_term["first_trade_date"])
df_yield["Date"] = pd.to_datetime(df_yield["Date"])

In [ ]:
# df_yield["SVENY05"] = df_yield["SVENY05"] / 100

In [ ]:
# rf_lookup = dict(zip(df_yield["Date"], df_yield["SVENY05"]))
# df_first_term["rfrate"] = df_first_term["first_trade_date"].map(rf_lookup)

In [ ]:
# lgd = 0.60
# df_first_term["hazardrate"] = 4 * np.log(1 + df_first_term["midspread"] / (4 * lgd))

# M = 5  # 5-year maturity
# j_vals = np.arange(1, 4 * M + 1)[:, None]  # Precompute j values
# hazard_exp = np.exp(-j_vals * df_first_term["hazardrate"].values / 4)  # Precompute e^(-jλ/4)
# rf_exp = np.exp(-j_vals * df_first_term["rfrate"].values / 4)  # Precompute e^(-jrf/4)

# df_first_term["RD"] = (1 / 4) * np.sum(hazard_exp * rf_exp, axis=0)

# df_first_term.head()

,ticker,redcode,month,first_trade_date,midspread,prev_spread,rfrate,hazardrate,RD
0,EMN,29EB75,2001-08-01 00:00:00-04:00,2001-08-10,0.011089,NaN,0.046233,0.018440,4.237669
1,CROATI,248DEC,2001-08-01 00:00:00-04:00,2001-08-01,0.016329,NaN,0.046636,0.027123,4.143190
2,TIN,8FGHEE,2001-05-01 00:00:00-04:00,2001-05-01,0.011838,NaN,0.049265,0.019682,4.192874
3,LNC,5I759F,2001-11-01 00:00:00-05:00,2001-11-01,0.004350,NaN,0.036626,0.007243,4.465046
4,CITOH,4ABC88,2001-07-01 00:00:00-04:00,2001-07-02,0.019080,NaN,0.049713,0.031674,4.066047


## Calculate RD

In [111]:
risk_free_df = df_yield[["SVENY01", "SVENY02", "SVENY03", "SVENY05", "SVENY07", "SVENY10"]].copy()
risk_free_df.columns = [1, 2, 3, 5, 7, 10]

In [116]:
risk_free_df

,1,2,3,5,7,10
0,2.9825,3.3771,3.5530,0.036987,3.7612,NaN
1,2.9941,3.4137,3.5981,0.037501,3.8154,NaN
2,3.0012,3.4142,3.5994,0.037531,3.8192,NaN
3,2.9949,3.4386,3.6252,0.037768,3.8418,NaN
4,2.9833,3.4101,3.5986,0.037533,3.8198,NaN
...,...,...,...,...,...,...
16613,NaN,NaN,NaN,NaN,NaN,NaN
16614,4.2622,4.2758,4.2977,0.043593,4.4350,4.5563
16615,4.2453,4.2504,4.2672,0.043260,4.4052,4.5365
16616,4.2366,4.2367,4.2495,0.043026,4.3785,4.5081


In [112]:
quarterly_maturities = np.arange(0.25, 10 + 0.25, 0.25)
interp_func = interp1d(
    risk_free_df.columns,  # Maturity years
    risk_free_df.iloc[0],  # Assuming we're using the first row; modify if needed
    kind="cubic", 
    fill_value="extrapolate"
)

In [113]:
interpolated_risk_free_rates = interp_func(quarterly_maturities)

In [ ]:
interpolated_rf_df = pd.DataFrame({
    "Maturity (Years)": quarterly_maturities,
    "Interpolated Rate": interpolated_risk_free_rates
})

In [115]:
interpolated_rf_df

,Maturity (Years),Interpolated Rate
0,0.25,NaN
1,0.50,NaN
2,0.75,NaN
3,1.00,NaN
4,1.25,NaN
5,1.50,NaN
6,1.75,NaN
7,2.00,NaN
8,2.25,NaN
9,2.50,NaN


## calculate CDS Returns

In [ ]:
cds_df.dropna()

,ticker,redcode,month,first_trade_date,midspread,prev_spread,rfrate,hazardrate,RD,prev_RD,CDS_Return
3457,AA,014B98,2001-02-01 00:00:00-05:00,2001-02-01,0.004079,0.004084,0.048552,0.006793,4.337667,4.339153,-0.000002
2032,AA,014B98,2001-03-01 00:00:00-05:00,2001-03-01,0.004099,0.004079,0.047025,0.006827,4.353978,4.337667,0.000103
6573,AA,014B98,2001-04-01 00:00:00-05:00,2001-04-02,0.003992,0.004099,0.046820,0.006648,4.358165,4.353978,-0.000450
826,AA,014B98,2001-05-01 00:00:00-04:00,2001-05-01,0.003231,0.003992,0.049265,0.005381,4.345294,4.358165,-0.003304
5692,AA,014B98,2001-06-01 00:00:00-04:00,2001-06-01,0.002508,0.003231,0.050176,0.004177,4.348490,4.345294,-0.003128
...,...,...,...,...,...,...,...,...,...,...,...
4035,ZLB,0G65DF,2001-06-01 00:00:00-04:00,2001-06-01,0.002692,0.002666,0.050176,0.004484,4.345148,4.355571,0.000123
4190,ZLB,0G65DF,2001-07-01 00:00:00-04:00,2001-07-02,0.002858,0.002692,0.049713,0.004760,4.347176,4.345148,0.000733
4003,ZLB,0G65DF,2001-08-01 00:00:00-04:00,2001-08-01,0.002875,0.002858,0.046636,0.004789,4.380636,4.347176,0.000086
5744,ZLB,0G65DF,2001-11-01 00:00:00-05:00,2001-11-01,0.003676,0.003506,0.036626,0.006122,4.477756,4.445057,0.000769


In [105]:
cds_df = df_first_term.copy()
cds_df = cds_df.sort_values(by=["ticker", "first_trade_date"])

cds_df["prev_spread"] = cds_df.groupby("ticker")["midspread"].shift(1)
cds_df["prev_RD"] = cds_df.groupby("ticker")["RD"].shift(1)

cds_df["CDS_Return"] = (cds_df["prev_spread"] / 250) + ((cds_df["midspread"] - cds_df["prev_spread"]) * cds_df["prev_RD"])

In [107]:
cds_df.dropna()

,ticker,redcode,month,first_trade_date,midspread,prev_spread,rfrate,hazardrate,RD,prev_RD,CDS_Return
3457,AA,014B98,2001-02-01 00:00:00-05:00,2001-02-01,0.004079,0.004084,0.048552,0.006793,4.337667,4.339153,-0.000002
2032,AA,014B98,2001-03-01 00:00:00-05:00,2001-03-01,0.004099,0.004079,0.047025,0.006827,4.353978,4.337667,0.000103
6573,AA,014B98,2001-04-01 00:00:00-05:00,2001-04-02,0.003992,0.004099,0.046820,0.006648,4.358165,4.353978,-0.000450
826,AA,014B98,2001-05-01 00:00:00-04:00,2001-05-01,0.003231,0.003992,0.049265,0.005381,4.345294,4.358165,-0.003304
5692,AA,014B98,2001-06-01 00:00:00-04:00,2001-06-01,0.002508,0.003231,0.050176,0.004177,4.348490,4.345294,-0.003128
...,...,...,...,...,...,...,...,...,...,...,...
4035,ZLB,0G65DF,2001-06-01 00:00:00-04:00,2001-06-01,0.002692,0.002666,0.050176,0.004484,4.345148,4.355571,0.000123
4190,ZLB,0G65DF,2001-07-01 00:00:00-04:00,2001-07-02,0.002858,0.002692,0.049713,0.004760,4.347176,4.345148,0.000733
4003,ZLB,0G65DF,2001-08-01 00:00:00-04:00,2001-08-01,0.002875,0.002858,0.046636,0.004789,4.380636,4.347176,0.000086
5744,ZLB,0G65DF,2001-11-01 00:00:00-05:00,2001-11-01,0.003676,0.003506,0.036626,0.006122,4.477756,4.445057,0.000769


In [99]:
cds_df[cds_df["ticker"] == "DB"]

,ticker,redcode,month,first_trade_date,midspread,prev_spread,rfrate,hazardrate,RD,prev_RD,CDS_Return
40,DB,2H6677,2001-11-01 00:00:00-05:00,2001-11-01,0.002504,NaN,0.036626,4.173691e-07,4.547998,NaN,NaN
55,DB,2H6677,2001-01-01 00:00:00-05:00,2001-01-03,0.002323,0.002504,0.050459,3.872351e-07,4.391299,4.547998,-0.000812
222,DB,2H6677,2001-12-01 00:00:00-05:00,2001-12-03,0.002756,0.002323,0.041796,4.593478e-07,4.488582,4.391299,0.001909
337,DB,2H6677,2001-06-01 00:00:00-04:00,2001-06-01,0.002326,0.002756,0.050176,3.876491e-07,4.394441,4.488582,-0.001920
393,DB,2H6677,2001-09-01 00:00:00-04:00,2001-09-03,0.002594,0.002326,NaN,4.323345e-07,NaN,4.394441,0.001188
804,DB,2H6677,2001-05-01 00:00:00-04:00,2001-05-01,0.002734,0.002594,0.049265,4.556728e-07,4.404542,NaN,NaN
2450,DB,2H6677,2001-08-01 00:00:00-04:00,2001-08-01,0.002538,0.002734,0.046636,4.229493e-07,4.433884,4.404542,-0.000854
2498,DB,2H6677,2001-07-01 00:00:00-04:00,2001-07-02,0.002635,0.002538,0.049713,4.391781e-07,4.399569,4.433884,0.000442
3106,DB,2H6677,2001-10-01 00:00:00-04:00,2001-10-01,0.002842,0.002635,0.039802,4.736643e-07,4.511375,4.399569,0.000921
3528,DB,2H6677,2001-02-01 00:00:00-05:00,2001-02-01,0.002494,0.002842,0.048552,4.156765e-07,4.412474,4.511375,-0.001558


## Construct Portfolio

In [108]:
cds_df = cds_df.sort_values(by=["month", "ticker"])
cds_df["spread_portfolio"] = cds_df.groupby("month")["midspread"].transform(
    lambda x: pd.qcut(x, q=20, labels=False, duplicates="drop") + 1  # Assigns portfolio labels 1-20
)

portfolio_returns = cds_df.groupby(["month", "spread_portfolio"])["CDS_Return"].mean().reset_index()
portfolio_returns 

,month,spread_portfolio,CDS_Return
0,2001-01-01 00:00:00-05:00,1,NaN
1,2001-01-01 00:00:00-05:00,2,NaN
2,2001-01-01 00:00:00-05:00,3,NaN
3,2001-01-01 00:00:00-05:00,4,NaN
4,2001-01-01 00:00:00-05:00,5,NaN
...,...,...,...
235,2001-12-01 00:00:00-05:00,16,-0.003653
236,2001-12-01 00:00:00-05:00,17,-0.001538
237,2001-12-01 00:00:00-05:00,18,0.001899
238,2001-12-01 00:00:00-05:00,19,-0.007485


In [ ]:
portfolio_returns_pivot = portfolio_returns.pivot(index="month", columns="spread_portfolio", values="CDS_Return")

# Rename columns to CDS1, CDS2, ..., CDS20
portfolio_returns_pivot.columns = [f"CDS{int(col)}" for col in portfolio_returns_pivot.columns]

portfolio_returns_pivot.iloc[1:]

,CDS1,CDS2,CDS3,CDS4,CDS5,CDS6,CDS7,CDS8,CDS9,CDS10,CDS11,CDS12,CDS13,CDS14,CDS15,CDS16,CDS17,CDS18,CDS19,CDS20
month,,,,,,,,,,,,,,,,,,,,
2001-02-01 00:00:00-05:00,-0.000034,-0.000453,-0.000063,-0.000212,-0.000892,-0.000745,-0.001062,-0.000415,-0.001474,-0.001447,-0.000140,-0.000831,-0.001752,-0.000782,-0.001901,-0.000243,0.002213,-0.001681,-0.007685,-0.037376
2001-03-01 00:00:00-05:00,-0.000177,-0.001145,0.000142,0.000588,-0.000431,-0.000462,-0.000095,-0.001011,-0.000017,-0.001796,0.000916,0.000558,0.000293,0.001756,-0.001704,0.001514,0.002901,0.001722,0.001290,0.002400
2001-04-01 00:00:00-05:00,-0.000283,0.000042,-0.000311,-0.000392,-0.001435,-0.000193,-0.001050,-0.001620,-0.000982,-0.000718,-0.000323,-0.000489,-0.003059,-0.003875,-0.002096,0.000422,-0.002841,0.000993,-0.000306,0.021189
2001-05-01 00:00:00-04:00,-0.000347,-0.000680,-0.001195,-0.001618,-0.001457,-0.001873,-0.002634,-0.001786,-0.002263,-0.001117,-0.002919,-0.002695,-0.002895,-0.005893,-0.005088,-0.005095,-0.006699,-0.003982,-0.000886,-0.011288
2001-06-01 00:00:00-04:00,-0.000288,-0.000531,-0.000502,-0.000194,-0.001375,-0.000407,-0.001186,-0.001305,-0.000886,-0.001600,-0.002517,-0.000539,-0.002455,-0.002667,-0.002980,-0.004002,-0.004137,-0.004532,0.002028,-0.015337
2001-07-01 00:00:00-04:00,-0.000301,-0.000400,-0.000160,-0.000099,-0.000002,-0.000363,-0.000304,-0.000006,0.000199,0.000400,-0.001679,0.000648,0.000011,0.001795,0.000668,-0.000187,0.002509,0.002543,0.007226,0.040260
2001-08-01 00:00:00-04:00,-0.000559,-0.000368,-0.000212,-0.000685,-0.000373,0.000150,-0.000652,0.000316,0.000064,-0.000546,-0.000339,-0.000526,-0.000511,-0.001427,0.000155,-0.001063,-0.000159,0.000133,-0.000344,0.005968
2001-09-01 00:00:00-04:00,-0.000020,0.000902,0.001285,0.001205,0.001470,0.001695,0.002507,0.002310,0.003158,0.004768,0.004713,0.003710,0.003718,0.006676,0.010293,0.009363,0.008863,0.012168,0.021294,0.059276
2001-10-01 00:00:00-04:00,0.001133,0.000442,0.000623,-0.001522,0.006977,0.001132,-0.000881,0.001872,0.001605,0.001386,0.010068,0.001635,0.007189,0.009219,0.008680,0.002062,-0.002911,0.024534,0.042920,0.014806
